<a href="https://colab.research.google.com/github/Kensuzuki95/GV918-Week04/blob/week-4/Week_04_Class_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

In this exercise, we will continue the analysis of UK Political Data, which I presented in the lecture. We have already conducted merging Election and Brexit referendum data. We will work with additional datasets, coming from: 

https://commonslibrary.parliament.uk/constituency-dashboard/

In particular, we will use the following data:

- Population
- Unemployment
- Housing price

# Clone a data repository

As we will use the data for several weeks, I decided to create a separate repository for the data. We can clone the reposiotory to Colab working directory to start working on the data.

In [8]:
!git clone  https://github.com/University-of-Essex-Dept-of-Government/GV918-UK-politics-data

Cloning into 'GV918-UK-politics-data'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


## Rerun the code in the lecture

In [9]:
import pandas as pd
import numpy as np


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:

df_elec = pd.read_excel("/content/GV918-UK-politics-data/Data/general-election-results-2019.xlsx", sheet_name = 'voting-summary')

df_brexit = pd.read_csv("/content/GV918-UK-politics-data/Data/brexit-vote.csv")

df_brexit.rename({'PCON11CD':'ons_id'}, axis = 1, inplace=True)

"""## Select columns"""

df_brexit.rename({'Figure to use':'leave_pct'}, axis = 1, inplace=True)

df_brexit_sub = df_brexit[['ons_id', 'leave_pct']]

df_elec['con_pct'] = df_elec['con'] / df_elec['valid_votes']
df_elec['lab_pct'] = df_elec['lab'] / df_elec['valid_votes']

df_elec.head()

"""## Merge"""
# use "left" merge

df_merge = df_elec.merge(df_brexit_sub, how='left', on = "ons_id")


# Unemployment

What we want to get from this file is the unemployment rate (or rate of unemployment claims) in November 2019, right before the election. 

## Read the file

In [24]:
df_unemp = pd.read_excel('/content/GV918-UK-politics-data/Data/Unemployment.xlsx', sheet_name = "Data")
df_unemp.head()

,ONSConstID,ConstituencyName,RegionID,RegionName,CountryID,CountryName,DateThisUpdate,DateOfDataset,UnempConstNumber,UnempConstRate,UnempRegionNumber,UnempRegionRate,UnempCountryNumber,UnempCountryRate,Unnamed: 14
0,E14000554,Berwick-upon-Tweed,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1216,0.025744,80105,0.04769,1502155,0.036923,0
1,E14000569,Bishop Auckland,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,2361,0.042521,80105,0.04769,1502155,0.036923,0
2,E14000574,Blaydon,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1909,0.034575,80105,0.04769,1502155,0.036923,0
3,E14000575,Blyth Valley,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,2248,0.042177,80105,0.04769,1502155,0.036923,0
4,E14000641,City of Durham,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1590,0.024718,80105,0.04769,1502155,0.036923,0


In [18]:
df_unemp.shape

(81250, 15)

In [19]:
df_unemp.columns

Index(['ONSConstID', 'ConstituencyName', 'RegionID', 'RegionName', 'CountryID',
       'CountryName', 'DateThisUpdate', 'DateOfDataset', 'UnempConstNumber',
       'UnempConstRate', 'UnempRegionNumber', 'UnempRegionRate',
       'UnempCountryNumber', 'UnempCountryRate', 'Unnamed: 14'],
      dtype='object')

In [26]:
df_unemp.value_counts('DateOfDataset').sort_index()

DateOfDataset
2010-05-01    650
2010-06-01    650
2010-07-01    650
2010-08-01    650
2010-09-01    650
             ... 
2020-05-01    650
2020-06-01    650
2020-07-01    650
2020-08-01    650
2020-09-01    650
Length: 135, dtype: int64

## Data wrangling and merge

- For this data, what we need to do is relatively simple
- The steps are:
  - Select the rows for the nearby month (Nov 2019)
  - Keep the columns for ID vars and unemployment rate
  - Merge with the main dataset

In [27]:
df_unemp[dfunemp['DateofDataset'] == "2020-11-01"][['ONSConstID', 'UnempConstRate']

SyntaxError: ignored

In [28]:
df_merge.merge(df_unemp_sub,left_on = 'ons-id', right_on = 'ONSConstID')

NameError: ignored

# Housing price dataset

We conduct essentially the same. Extract the most recent data in housing price, and merge it with the main dataframe.

## Read the file

In [13]:
df_housing = pd.read_excel("/content/GV918-UK-politics-data/Data/House-prices.xlsx", 
                           sheet_name="Constituency data table")

## Data wrangling and merge

- First let's findout the data frequency. 

In [30]:
df_housing.value_counts('DateOfDataset').sort_index()

DateOfDataset
1995-12-31    650
1996-03-31    650
1996-06-30    650
1996-09-30    650
1996-12-31    650
             ... 
2018-12-01    650
2019-03-01    650
2019-06-01    650
2019-09-01    650
2019-12-01    650
Length: 97, dtype: int64

- Select the rows from Sep 2019
- We use the price change at the constituency level

## Population dataset

What we would like to extract from this dataset is the percentage of elderly (over 65). We need to do some data wrangling.

## Read the dataset

In [14]:
df_pop = pd.read_excel("/content/GV918-UK-politics-data/Data/population-by-age.xlsx", 
                       sheet_name = 'Age by year data')

In [31]:
df_pop.head()

,PCON11CD,PCON11NM,RegionID,RegionNM,All Ages,Age_year,Age_pop,Age_percent,Reg_percent,UK_percent
0,E14000538,Aylesbury,E12000008,South East,125629,0,1601,0.012744,0.010438,0.010822
1,E14000538,Aylesbury,E12000008,South East,125629,1,1664,0.013245,0.010968,0.011266
2,E14000538,Aylesbury,E12000008,South East,125629,2,1772,0.014105,0.011386,0.011637
3,E14000538,Aylesbury,E12000008,South East,125629,3,1775,0.014129,0.011892,0.012012
4,E14000538,Aylesbury,E12000008,South East,125629,4,1793,0.014272,0.012033,0.012009


## Data wrangling

- This data has so many rows, but suppose that what we want to know is the population over the age of 65.

- We will clean the data using the following steps
  - Keep rows of year over 65
  - Aggregate the percentage of the population


In [32]:
df_pop_sub = df_pop[df_pop['Age_year'] >65]

In [39]:
df_elderly = df_pop_sub[['PCON11CD', 'Age_percent']].groupby('PCON11CD').sum().reset_index()

## Merge with the main dataset

In [40]:
df_merge = df_merge.merge(df_elderly, left_on = 'ons_id', right_on = 'PCON11CD')

In [42]:
df_merge.head()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,declaration_time,mp_firstname,mp_surname,mp_gender,result,first_party,second_party,electorate,valid_votes,invalid_votes,majority,con,lab,ld,brexit,green,snp,pc,dup,sf,sdlp,uup,alliance,other,other_winner,mp_fullname,majority_percent,majority_rank,turnout_const,turnout_reg,turnout_uk,turnout_2017,con_pct,lab_pct,leave_pct,PCON11CD,Age_percent
0,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,2019-12-13 02:30:00,Stephen,Kinnock,Male,Lab hold,Lab,Con,50750,31598,82,10490,6518,17008,1072,3108,450,0,2711,0,0,0,0,0,731,0,Stephen Kinnock,0.331983,209,0.622621,0.665721,0.673108,0.666800,0.206279,0.538262,0.601245,W07000049,0.185279
1,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,2019-12-13 03:09:00,Robin,Millar,Male,Con hold,Con,Lab,44699,31865,123,2034,14687,12653,1821,0,0,0,2704,0,0,0,0,0,0,0,Robin Millar,0.063832,569,0.712879,0.665721,0.673108,0.710482,0.460913,0.397081,0.521971,W07000058,0.260642
2,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,2019-12-13 03:13:00,Kirsty,Blackman,Female,SNP hold,SNP,Con,62489,37413,72,12670,7535,4939,2846,1008,880,20205,0,0,0,0,0,0,0,0,Kirsty Blackman,0.338652,202,0.598713,0.680736,0.673108,0.591614,0.201401,0.132013,0.430922,S14000001,0.128157
3,S14000002,S92000003,Aberdeen South,Scotland,Scotland,Scotland,Borough,2019-12-13 03:34:00,Stephen,Flynn,Male,SNP gain from Con,SNP,Con,65719,45638,131,3990,16398,3834,5018,0,0,20388,0,0,0,0,0,0,0,0,Stephen Flynn,0.087427,533,0.694441,0.680736,0.673108,0.684887,0.359306,0.084009,0.321431,S14000002,0.152222
4,S14000003,S92000003,Airdrie and Shotts,Scotland,Scotland,Scotland,County,2019-12-13 02:59:00,Neil,Gray,Male,SNP hold,SNP,Lab,64011,39772,91,5201,7011,12728,1419,0,685,17929,0,0,0,0,0,0,0,0,Neil Gray,0.130770,459,0.621331,0.680736,0.673108,0.592430,0.176280,0.320024,0.398381,S14000003,0.160167


# Check correlation

- Let's check the correlation between `con_pct`, `lab_pct`, `leave_pct` and merged variables

In [45]:
df_merge.loc[:, ['con_pct', 'lab_pct', 'leave_pct', 'Age_percent']]

,con_pct,lab_pct,leave_pct,Age_percent
0,0.206279,0.538262,0.601245,0.185279
1,0.460913,0.397081,0.521971,0.260642
2,0.201401,0.132013,0.430922,0.128157
3,0.359306,0.084009,0.321431,0.152222
4,0.176280,0.320024,0.398381,0.160167
...,...,...,...,...
645,0.300699,0.532965,0.496481,0.140722
646,0.583665,0.063466,0.598655,0.229158
647,0.354536,0.300695,0.509420,0.247519
648,0.278093,0.551702,0.388207,0.126830
